# A basic ML model to check the freshness of a fruit

## Importing Libraries

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Data preprocessing

### Setting Directories

- **"tf.keras.utils.image_dataset_from_directory"** is a library function which loads images from a directory
- we have three directories here, *test, train and validation*, each directory here has four classes
- label_mode defines how the lables are represented by the library function
- seed ensures the random split between training and validation dataset is consistent and no images appear in both subsets if you run your code multiple times. (since we are splitting training set into two parts instead of creating a different directory for validation set)

In [5]:
batch_size = 32 #instead of processing one image at a time, the images are processed in a batch of 32
img_height = 180
img_width = 180
validation_split = 0.2

train_ds = tf.keras.utils.image_dataset_from_directory(
    r"D:\Study stuff\Fruit insights project\model\Model-1-fruit-freshness\train",
    validation_split=validation_split,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical',  
    shuffle=True,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    r"D:\Study stuff\Fruit insights project\model\Model-1-fruit-freshness\train",
    validation_split=validation_split,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=True,
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    r"D:\Study stuff\Fruit insights project\model\Model-1-fruit-freshness\test",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical',  
    shuffle=False,
)


Found 8733 files belonging to 4 classes.
Using 6987 files for training.
Found 8733 files belonging to 4 classes.
Using 1746 files for validation.
Found 2570 files belonging to 4 classes.


### Normalizing pixel values

- Raw images have pixel values in range 0-255
- The neural networks train faster in a standardized range (0.0 - 1.0). For that we use normalization, it reduces effects of variations in lightning, brightness and contrast in images.
- **normalization_layer** rescales each pixel by multiplying it from (1/255) which converts the range of pixels to a floating point number between 0.0 to 1.0.

---

- here the **map()** applies the lambda function to each batch
- **x** is a batch of images
- **y** is a batch of labels for each image in the batch (categories of images i.e. fresh apples, fresh oranges etc)
- you can imagine labels are [0,1,2,3] where 0 is for fresh apples, 1 is for fresh oranges, 2 is for rotten apples and 3 is for rotten oranges. The labels remain the same.


In [6]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

### Optimizing data set loading

- **tf.data.AUTOTUNE** makes sure the data pipeline has the right number of threads working in parallel so training is smooth and efficient. Here threads are for the number of operations like map(), prefetch() etc. that are present in the data pipeline.
- **data pipeline** includes a number of processes such as, loading them from the disk, resizing them, shuffling them, normalizing them, batch them and feed them to the model.
- **cache()**: Stores the data in RAM after the first epoch (or on disk if RAM is small), this avoids reloading images again in later epochs which results in faster training.
- **shuffle():** is used because it randomizes the order of images so that the machine is not focusing on a specific order during training, tensorflow randomly selects the next image from the buffer.
- **prefetch()**: prepares the next batch of data while the model is being trained on the current one
- **prefetch(buffer_size=AUTOTUNE)**: lets tensorflow automatically choose the optimal number of batches to prepare in advance.

In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


## CNN model
( Convolutional Neural Network )

- The **layers module** provides building blocks of a neural network. Here each layer performs a specific transformation on data.
Such as - **Conv2D** : detects patterns/features
        - **MaxPooling2D** : shrinks data while keeping important information
        - **Flatten** : Converts Multidimentional data into 1D vector
        - **Dense** : Combines all features to make a decision

- The **models module** provides a container to organize and connect the layers together into a complete neural network

In [8]:
from tensorflow.keras import layers, models

### Model creation

I am using a **Sequential model**, which means the output of one layer is directly passed as input to the next (layers are stacked one after another). The steps are given below. 

#### 1. Input Layer
- The model takes an image of size **180 × 180** with **3 color channels (RGB)**.

#### 2. Convolution + Pooling Layers (Feature Extraction)
- **Conv2D(32, 3×3, ReLU):** Detects very basic features like *edges, corners, and lines*. 
- **ReLU:** ensures we only keep the useful signals by removing negative values
- **MaxPooling(2×2):** Shrinks the image while keeping the strongest signals. This process makes the model faster and avoids overfitting.
- **Dropout(0.25):** Dropout randomly sets 25% of those values to 0. This prevents the model to memorize which helps to focus on learning different patterns.  

- **Conv2D(64, 3×3, ReLU):** Detects more complex features like *textures, curves, and shapes*.  
- **MaxPooling(2×2):** Same function
- **Dropout(0.25):** Same function

- **Conv2D(128, 3×3, ReLU):** Detects high-level, detailed features like *eyes, leaves, or object parts*.  
- **MaxPooling(2×2):** Same function. 
- **Dropout(0.25):** Same function

At this stage, the image has been turned into a set of **2D feature maps** (many small grids that represent different learned patterns).

#### 3. Flatten Layer
- The 2D feature maps are **flattened into a 1D vector**.  
- This means all the detected features are lined up in a row so they can be used for decision making.

#### 4. Dense(128, ReLU)
- This is a **fully connected layer** that learns **combinations of features**.  
- Example: *“If this edge + this texture + this color appear together  :- it might mean ‘fresh fruit’.”*  
- **Dropout(0.50):** In dense layers every neuron is connected to every neuron in next layer so it makes them prone to *overfitting* (memorizing), therefore, dropping 50% of neurons ensures that model focuses on general patterns instead of memorizing.

#### 5. Dense(4, Softmax)
- This layers has 4 neurons (because we have 4 classes). So each neuron corresponds to one class.
- The output comes out as raw values called logits, they can be positive or negative.
- Softmax converts them into probabilities that add up to 1. 
- For example if logits are [-0.35, 0.1, 3.4, 0.01]
- then after softmax it will be [0.8, 0.1, 0.05, 0.05] (sum of them is equal to 1). This means that the machine is 80% confident that the given image belongs to first class.

In conclusion, Convolution + pooling are feature extracters, the dense layers act as decision makers, softmax gives us probabilites for each class.


In [9]:
model = models.Sequential([
    layers.Input(shape=(180, 180, 3)),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(4, activation='softmax')  
])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 178, 178, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 87, 87, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 41, 41, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 51200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     6,553,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,647,492 (25.36 MB)

 Trainable params: 6,647,492 (25.36 MB)

 Non-trainable params: 0 (0.00 B)

### Compilation of the model

- Here the neural network gets ready for training
- **optimizer** is like an engine of learning.
- **'adam'** (Adaptive moment estimation) automatically updates the learning rate for each parameter so the model continues to learn from it's mistakes.
- The **loss function** measures how far the model's predictions are from the actual data.
- **Categorical crossentropy** focuses on the correct class and checks how confident the model is about predicting it.
- **metrics** tell us how well the model is performing, accuracy simply means the percentage of correct prediction.

In [10]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

### Training the model

- **EarlyStopping** is used to prevent overfitting. It will watch the validation loss and stops training when there is no improvement.
- **patience = 2** means it will wait for 2 epochs to see if the validation loss improves and if it doesn't then it will stop the training.
- **restore_best_weights=True** makes sure the best version of the model is kept
---
- **epochs** means that the model will go through the entire training dataset 10 times and each time it will try to improve.
---
- **model.fit** trains the neural network, it takes the training dataset in batches and feeds it to the model.
- After each epoch, the model will check its perfomance on validation data to see if it's learning patterns or overfitting(memorizing).
- Normally it will run for 10 epochs. Where first epoch will be 1/10 and last will be 10/10.
- But with early_stop the training may stop before 10 epochs if the validation loss is not improving.
- This training process is saved in "history"
---
The training process is tracked with:
- accuracy → how well it predicts on training data
- loss → error on training data
- val_accuracy → how well it predicts on validation data
- val_loss → error on validation data
---
***Why are we checking validation loss?***
The model sees training data again and again so it might memorize it. Validation data is the unseen data so with that data we check if our model is learning or memorizing. By improvement in validation loss value, we mean the lower it gets the better. When the validation loss starts going up, that's when the earlyStopping kicks in. If the validation loss stops improving it means that the model is memorizing training data and it's not generalizing well.

***Accuracy vs loss***
- Accuracy is a percentage of correctly predicted labels.
- Loss is a numerical score which tells how wrong the predictions are. 
- We are using *categorical crossentrophy* which looks at the predicted probabilities (from softmax). The higher the probability for correct class, smaller the loss. vice versa.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping 

early_stop = EarlyStopping(monitor='val_loss',patience = 2, restore_best_weights=True)

epochs = 10

history = model.fit(
    train_ds,
    validation_data=val_ds,  
    epochs=epochs,
    callbacks=[early_stop]
)

Epoch 1/10
 98/219 ━━━━━━━━━━━━━━━━━━━━ 3:30 2s/step - accuracy: 0.3936 - loss: 1.6507

### Analysis

In [2]:
import matplotlib.pyplot as plt

# collecting the data we received from model training for each epoch
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs) # used as x-axis values

plt.figure(figsize=(12, 6)) # figure size , 12 width and 6 height
plt.subplot(1, 2, 1) # 1 row, 2 columns, index (first subplot)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2) # 1 row, 2 columns, index (second subplot)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


NameError: name 'history' is not defined

> So from the above plots let's understand what we got.
- **Training and Validation accuracy**: Both curves are steadily increasing and validation accuracy follows training accuracy closely which means the model is generalizing well (not overfitting). The accuracy reaches close to 95%.
- **Training and Validation loss**: Both losses are decreasing nicely. Since validation loss continues to go down (instead of rising up in the end), it means that the model is effectively.

### Evaluation

- **model.evaluate(test_ds)** : It runs the model in test dataset using the same metrics we defined during compilation (loss and accuracy). It returns them as a list [loss, accuracy]
- **test_loss** : error score on test dataset
- **test_acc** : percentage of correct predictions on test dataset
- **print(f"Test accuracy: {test_acc:.3f}")** : prints accuracy rounded to 3 decimal places

In [1]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.3f}")

NameError: name 'model' is not defined

### Make predictions

In [ ]:
for images, labels in test_ds.take(1):  # Takes one batch from test data
    predictions = model.predict(images)
    print("Predicted class (first 5):", predictions.argmax(axis=1)[:5])
    print("True class (first 5):", labels.numpy().argmax(axis=1)[:5])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step
Predicted class (first 5): [0 0 0 0 0]
True class (first 5): [0 0 0 0 0]


### Save model

In [16]:
model.save('fruit_freshness_cnn_model.h5')
